In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
def load_binary_classif():
    from sklearn.datasets import load_breast_cancer
    from sklearn.preprocessing import StandardScaler
    X = load_breast_cancer().data
    y = load_breast_cancer().target
    st = StandardScaler()
    st.fit(X)
    X = st.transform(X)
    return X, y

def load_multiclassif():
    mnist = tf.keras.datasets.mnist
    (xtrain, ytrain), (xtest, ytest) = mnist.load_data()
    xtrain = xtrain / 255.
    xtest = xtest / 255.
    return xtrain, ytrain, xtest, ytest

In [3]:
class ANN():
    
    def __init__(self, layers, activation_function, kind, num_classes = None):
        """
        layers = list with the number of neurons per layers / the length of the list is the amount of hidden layers
        activation_function = from keras;
        kind = ['regression', 'binary_classif','multiclass_classif']
        """
        self.layers = layers
        self.activation_function = activation_function
        self.kind = kind
        self.num_classes = num_classes
        self.model = None
        self.res = None
    
    def fit(self, X, y, epochs, val_split, optimizer, metrics_tracking):
        D = X.shape[1]
        input_layer = tf.keras.layers.Input(shape = (D,))
        hidden_layers = [tf.keras.layers.Dense(x, activation = self.activation_function) for x in self.layers]
        
        if self.kind == 'regression':
            output_layer = tf.keras.layers.Dense(1)
            model = tf.keras.models.Sequential([input_layer] + hidden_layers + [output_layer])
            model.compile(optimizer = optimizer, 
                          loss = 'mse', 
                          metrics = metrics_tracking)
            
        if self.kind == 'binary_classif':
            output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')
            model = tf.keras.models.Sequential([input_layer] + hidden_layers + [output_layer])
            model.compile(optimizer = optimizer, 
                          loss = 'binary_crossentropy', 
                          metrics = metrics_tracking)
            
        if self.kind == 'multiclass_classif':
            output_layer = tf.keras.layers.Dense(self.num_classes, activation = 'softmax')
            model = tf.keras.models.Sequential([input_layer] + hidden_layers + [output_layer])
            model.compile(optimizer = optimizer, 
                          loss = 'sparse_categorical_crossentropy', 
                          metrics = metrics_tracking)

        res = model.fit(X, y, epochs = epochs, validation_split = val_split)
        self.res = res
        self.model = model
        
    def predict(self, X):
        if self.model == None:
            print('Train it first.')
        else:
            return self.model.predict(X)

In [4]:
X, y = load_binary_classif()

In [5]:
ann = ANN(layers = [10,10,10], activation_function = 'relu', kind = 'binary_classif')

ann.fit(X, 
        y, 
        epochs = 10, 
        val_split = 0.33, 
        optimizer = tf.keras.optimizers.Adam(), 
        metrics_tracking = [tf.keras.metrics.binary_crossentropy])

Epoch 1/10
12/12 [==============================] - 1s 17ms/step - loss: 0.6769 - binary_crossentropy: 0.6769 - val_loss: 0.7697 - val_binary_crossentropy: 0.7697
Epoch 2/10
12/12 [==============================] - 0s 4ms/step - loss: 0.6090 - binary_crossentropy: 0.6090 - val_loss: 0.6973 - val_binary_crossentropy: 0.6973
Epoch 3/10
12/12 [==============================] - 0s 4ms/step - loss: 0.5598 - binary_crossentropy: 0.5598 - val_loss: 0.6418 - val_binary_crossentropy: 0.6418
Epoch 4/10
12/12 [==============================] - 0s 4ms/step - loss: 0.5205 - binary_crossentropy: 0.5205 - val_loss: 0.5961 - val_binary_crossentropy: 0.5961
Epoch 5/10
12/12 [==============================] - 0s 4ms/step - loss: 0.4833 - binary_crossentropy: 0.4833 - val_loss: 0.5545 - val_binary_crossentropy: 0.5545
Epoch 6/10
12/12 [==============================] - 0s 4ms/step - loss: 0.4464 - binary_crossentropy: 0.4464 - val_loss: 0.5139 - val_binary_crossentropy: 0.5139
Epoch 7/10
12/12 [=========

In [13]:
#ann.predict(X)

In [7]:
Xtrain, ytrain, Xtest, ytest = load_multiclassif()
Xtrain = Xtrain.reshape(Xtrain.shape[0], -1)
Xtest = Xtest.reshape(Xtest.shape[0], -1)

In [8]:
len(np.unique(y))

2

In [9]:
ann = ANN(layers = [10,10,10], activation_function = 'relu', kind = 'multiclass_classif', num_classes = 10)

ann.fit(Xtrain, 
        ytrain, 
        epochs = 10, 
        val_split = 0.33, 
        optimizer = tf.keras.optimizers.Adam(), 
        metrics_tracking = [tf.keras.metrics.sparse_categorical_crossentropy])

Epoch 1/10
1257/1257 [==============================] - 3s 2ms/step - loss: 0.8310 - sparse_categorical_crossentropy: 0.8310 - val_loss: 0.4592 - val_sparse_categorical_crossentropy: 0.4592
Epoch 2/10
1257/1257 [==============================] - 2s 2ms/step - loss: 0.3926 - sparse_categorical_crossentropy: 0.3926 - val_loss: 0.3510 - val_sparse_categorical_crossentropy: 0.3510
Epoch 3/10
1257/1257 [==============================] - 2s 2ms/step - loss: 0.3289 - sparse_categorical_crossentropy: 0.3289 - val_loss: 0.3124 - val_sparse_categorical_crossentropy: 0.3124
Epoch 4/10
1257/1257 [==============================] - 2s 2ms/step - loss: 0.2952 - sparse_categorical_crossentropy: 0.2952 - val_loss: 0.3064 - val_sparse_categorical_crossentropy: 0.3064
Epoch 5/10
1257/1257 [==============================] - 2s 2ms/step - loss: 0.2736 - sparse_categorical_crossentropy: 0.2736 - val_loss: 0.2813 - val_sparse_categorical_crossentropy: 0.2813
Epoch 6/10
1257/1257 [============================

In [10]:
ann.predict(Xtest).shape

313/313 [==============================] - 0s 917us/step


(10000, 10)

In [11]:
ann.predict(Xtest)

313/313 [==============================] - 0s 876us/step


array([[2.46936077e-04, 1.73639671e-06, 1.12590868e-04, ...,
        9.65054274e-01, 1.00849356e-05, 2.47419681e-02],
       [1.29745911e-06, 1.05890686e-04, 9.97537374e-01, ...,
        4.24859259e-10, 1.21783273e-07, 1.33589049e-13],
       [2.93655394e-05, 9.87826884e-01, 1.60749757e-03, ...,
        6.96658448e-04, 4.23276704e-03, 1.32338464e-04],
       ...,
       [6.08946635e-08, 5.51149640e-07, 3.00366878e-08, ...,
        1.64962174e-07, 3.45369405e-03, 4.10738140e-02],
       [6.77468790e-07, 5.63107431e-04, 3.88226985e-11, ...,
        4.56900989e-10, 2.11073179e-03, 1.08636759e-05],
       [7.96527456e-06, 1.69712075e-05, 3.54338299e-05, ...,
        1.95011549e-10, 6.00837484e-05, 6.25480290e-09]], dtype=float32)

In [12]:
preds = np.argmax(ann.predict(Xtest), axis = 1)

acc__ = sum(preds == ytest) / len(ytest)
print('test acc:', acc__)

313/313 [==============================] - 0s 914us/step
test acc: 0.9255
